In [73]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


import warnings
warnings.filterwarnings("ignore")

In [118]:
# params < 20k ;  
# BN, Dropout, a Fully connected layer, GAP; 
# < 20 epochs;
#  > 99.4
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) # 30*30*1 | 3*3 | 28*28*32
        self.bn1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)  # 30*30*32 | 3*3 | 28*28*64
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) # 30*30*32 | 3*3 | 28*28*64 # 16*16*64 | 3*3 | 14*14*128
        self.bn3 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # 28*28*64 | 14*14*64
        self.conv31 = nn.Conv2d(32, 8, 1, padding=1)  # 14*14*64| 14*14*8 # 16*16*64 | 3*3 | 14*14*128
        
        self.conv4 = nn.Conv2d(8, 16, 3, padding=1)  # 14*14*8  | 12  # 30*30*32 | 3*3 | 28*28*64
        self.bn4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1) # 30*30*32 | 3*3 | 28*28*64 # 16*16*64 | 3*3 | 14*14*128
        self.bn5 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)  # 7# 28*28*64 | 14*14*64
        self.conv51 = nn.Conv2d(32, 8, 1, padding=1)  # 7*7*8 # 14*14*64| 14*14*8 # 16*16*64 | 3*3 | 14*14*128

        self.conv6 = nn.Conv2d(8, 16, 3, padding=1)  # 16*16*128 | 3*3 | 14*14*256
        self.bn6 = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 32, 3, padding=1)  # 16*16*128 | 3*3 | 14*14*256
        self.bn7 = nn.BatchNorm2d(32)

        self.gap1 =  nn.AvgPool2d(7) # 7*7*256 | 1*1*256  # 1*1*32
        self.fc1 = nn.Linear(32, 10)  # 32

    def forward(self, x):
        x = self.conv31(self.pool1(self.bn3(F.relu(self.conv3(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))))))
        # print(x.shape)
        x = self.conv51(self.pool1(self.bn5(F.relu(self.conv5(self.bn4(F.relu(self.conv4(x))))))))
        # print(x.shape)
        x = (self.bn7(F.relu(self.conv7(self.bn6(F.relu(self.conv6(x)))))))
        x = self.gap1(x)
        # print(x.shape)
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = (self.fc1(x))
        return F.log_softmax(x)

In [119]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
            Conv2d-5           [-1, 32, 28, 28]           4,640
       BatchNorm2d-6           [-1, 32, 28, 28]              64
         MaxPool2d-7           [-1, 32, 14, 14]               0
            Conv2d-8            [-1, 8, 16, 16]             264
            Conv2d-9           [-1, 16, 16, 16]           1,168
      BatchNorm2d-10           [-1, 16, 16, 16]              32
           Conv2d-11           [-1, 32, 16, 16]           4,640
      BatchNorm2d-12           [-1, 32, 16, 16]              64
        MaxPool2d-13             [-1, 32, 8, 8]               0
           Conv2d-14            [-1, 8,

In [76]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [123]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    # pbar = tqdm(train_loader)
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    train_loss /= len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [124]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print('epoch number ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch number  1
Train set: Average loss: 0.0037, Accuracy: 51360/60000 (85.60%)
Test set: Average loss: 0.0925, Accuracy: 9737/10000 (97.37%)

epoch number  2
Train set: Average loss: 0.0006, Accuracy: 58721/60000 (97.87%)
Test set: Average loss: 0.0540, Accuracy: 9846/10000 (98.46%)

epoch number  3
Train set: Average loss: 0.0004, Accuracy: 59146/60000 (98.58%)
Test set: Average loss: 0.0452, Accuracy: 9867/10000 (98.67%)

epoch number  4
Train set: Average loss: 0.0003, Accuracy: 59296/60000 (98.83%)
Test set: Average loss: 0.0436, Accuracy: 9869/10000 (98.69%)

epoch number  5
Train set: Average loss: 0.0003, Accuracy: 59450/60000 (99.08%)
Test set: Average loss: 0.0407, Accuracy: 9863/10000 (98.63%)

epoch number  6
Train set: Average loss: 0.0002, Accuracy: 59513/60000 (99.19%)
Test set: Average loss: 0.0408, Accuracy: 9863/10000 (98.63%)

epoch number  7
Train set: Average loss: 0.0002, Accuracy: 59511/60000 (99.19%)
Test set: Average loss: 0.0327, Accuracy: 9903/10000 (99.03%)
